In [6]:
import pandas as pd
from auxiliary.paths import input_path, raw_path
from astropy.table import Table
import numpy as np
from auxiliary.columns import wise, galex, create_colors, calculate_colors, splus, error_splus
from auxiliary.correct_extinction import correction
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
filename = "/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_dual_VAC_features.fits"
table = Table.read(os.path.join(input_path, filename))
table = table.to_pandas()
table["ID"] = table["ID"].str.decode('utf-8') 

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_dual_VAC_features.fits'

In [ ]:
table['W1_MAG'] = 22.5 - 2.5*np.log10(table['FW1']) # + 2.699
table['W2_MAG'] = 22.5 - 2.5*np.log10(table['FW2']) # + 3.339

# http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1image.html#a0a023f269211d52086723788764d484e
# return std::abs(fluxErr / (-0.4 * flux * std::log(10)));
table['e_W1_MAG'] = abs(table['e_FW1'] / (-0.4*table['FW1']*np.log(10)))
table['e_W2_MAG'] = abs(table['e_FW2'] / (-0.4*table['FW2']*np.log(10)))

table['W1_MAG'].replace([np.inf, -np.inf], np.nan, inplace=True)
table['W2_MAG'].replace([np.inf, -np.inf], np.nan, inplace=True)

/Users/Mariko/opt/anaconda3/envs/photoz/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Mariko/opt/anaconda3/envs/photoz/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
table[wise+galex] = table[wise+galex].fillna(value=99)

In [ ]:
correct = False

if correct:
    table = correction(table)
calculate_colors(table, broad=True, narrow=True, wise=True, galex = True, aper="PStotal")
features = create_colors(broad=True, narrow=True, wise=True, galex=True, aper="PStotal")
pos_splus = ["ID", "RA", "DEC"]

if correct:
    table[pos_splus+features+splus+wise+galex+error_splus].to_csv("/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_QSOz_VAC_ext.csv", index=False)
else:
    table[pos_splus+features+splus+wise+galex+error_splus].to_csv("/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_QSOz_VAC.csv", index=False)

In [12]:
corrected_df.dropna(subset=splus)
corrected_df.query(f"ID=='iDR4_3_STRIPE82-0170_0022523'")[splus]

,u_PStotal,g_PStotal,r_PStotal,i_PStotal,z_PStotal,J0378_PStotal,J0395_PStotal,J0410_PStotal,J0430_PStotal,J0515_PStotal,J0660_PStotal,J0861_PStotal
22522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
features

['u_PStotal-r_PStotal',
 'g_PStotal-r_PStotal',
 'r_PStotal-i_PStotal',
 'r_PStotal-z_PStotal',
 'J0378_PStotal-r_PStotal',
 'J0395_PStotal-r_PStotal',
 'J0410_PStotal-r_PStotal',
 'J0430_PStotal-r_PStotal',
 'J0515_PStotal-r_PStotal',
 'r_PStotal-J0660_PStotal',
 'r_PStotal-J0861_PStotal',
 'r_PStotal-W1_MAG',
 'r_PStotal-W2_MAG',
 'FUVmag-r_PStotal',
 'NUVmag-r_PStotal']

In [12]:
Magnitudes

['FUVmag',
 'NUVmag',
 'u_PStotal',
 'g_PStotal',
 'r_PStotal',
 'i_PStotal',
 'z_PStotal',
 'J0378_PStotal',
 'J0395_PStotal',
 'J0410_PStotal',
 'J0430_PStotal',
 'J0515_PStotal',
 'J0660_PStotal',
 'J0861_PStotal',
 'W1_MAG',
 'W2_MAG']

In [17]:
features

['u_PStotal-r_PStotal',
 'g_PStotal-r_PStotal',
 'r_PStotal-i_PStotal',
 'r_PStotal-z_PStotal',
 'J0378_PStotal-r_PStotal',
 'J0395_PStotal-r_PStotal',
 'J0410_PStotal-r_PStotal',
 'J0430_PStotal-r_PStotal',
 'J0515_PStotal-r_PStotal',
 'r_PStotal-J0660_PStotal',
 'r_PStotal-J0861_PStotal',
 'r_PStotal-W1_MAG',
 'r_PStotal-W2_MAG',
 'FUVmag-r_PStotal',
 'NUVmag-r_PStotal']

In [15]:
Dataframe = table.copy(deep=True)
Magnitudes = galex+splus+wise
Aper = "PStotal"
Reference_Mag  = 'r_'+Aper
Reference_Idx = Magnitudes.index(Reference_Mag)
MagnitudesToLeft = Magnitudes[:Reference_Idx]
MagnitudesToRight = Magnitudes[(Reference_Idx+1):]

for feature in MagnitudesToLeft: # of Reference_Mag
    Dataframe[feature+'-'+Reference_Mag] = Dataframe[feature] - Dataframe[Reference_Mag]

for feature in MagnitudesToRight: # of Reference_Mag
    Dataframe[Reference_Mag+'-'+feature] = Dataframe[Reference_Mag] - Dataframe[feature]

Training_Features = []
for s in Dataframe.columns.values:
    if '-' in s:
        if s.split('-')[0] in Magnitudes and s.split('-')[1] in Magnitudes:
            Training_Features.append(s)

In [16]:
Dataframe.columns.values

array(['ID', 'PHOT_ID_dual_1', 'RA', 'DEC', 'A', 'B_in', 'KRON_RADIUS',
       'FWHM_n', 'MU_MAX_INST', 'ISOarea', 'SEX_FLAGS_DET',
       's2n_DET_aper_6', 'PHOT_ID_dual_2', 'SEX_FLAGS_u', 'u_iso',
       'e_u_iso', 'u_aper_3', 'e_u_aper_3', 'u_aper_6', 'e_u_aper_6',
       'u_auto', 'e_u_auto', 'u_petro', 'e_u_petro', 'u_PStotal',
       'e_u_PStotal', 'FWHM_n_u', 'MU_MAX_u', 'PHOT_ID_dual_3',
       'SEX_FLAGS_J0378', 'J0378_iso', 'e_J0378_iso', 'J0378_aper_3',
       'e_J0378_aper_3', 'J0378_aper_6', 'e_J0378_aper_6', 'J0378_auto',
       'e_J0378_auto', 'J0378_petro', 'e_J0378_petro', 'J0378_PStotal',
       'e_J0378_PStotal', 'FWHM_n_J0378', 'MU_MAX_J0378',
       'PHOT_ID_dual_4', 'SEX_FLAGS_J0395', 'J0395_iso', 'e_J0395_iso',
       'J0395_aper_3', 'e_J0395_aper_3', 'J0395_aper_6', 'e_J0395_aper_6',
       'J0395_auto', 'e_J0395_auto', 'J0395_petro', 'e_J0395_petro',
       'J0395_PStotal', 'e_J0395_PStotal', 'FWHM_n_J0395', 'MU_MAX_J0395',
       'PHOT_ID_dual_5', 'SEX_FLAGS_J

In [23]:
import os
import glob
import pickle
import pandas as pd
import numpy as np
from joblib import load
from auxiliary.paths import results_path,  input_path, model_path, raw_path
from bmdn.bmdn_predict import FinalPredict, Process_Final
from tensorflow.keras.models import load_model
from auxiliary.correct_extinction import correction 
from auxiliary.columns import create_colors
from auxiliary.crossmatch import match_stilts, process_data


In [9]:
list_raw = glob.glob(os.path.join(raw_path, "*.fits"))


In [16]:
list_raw[0].split(os.path.sep)[-1].split('_')[0]+"_QSOs_VAC.csv"

'STRIPE82-0170_QSOs_VAC.csv'

In [24]:
for file in list_raw:
    # print(file)
    match_stilts(file)

In [31]:
list_input = glob.glob(os.path.join(input_path, "*.fits"))

for file in list_input:
    process_data(file, correct_ext = False, save=True)

/Users/astgst/opt/anaconda3/envs/lili/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [32]:
test = pd.read_table("/Users/astgst/Documents/Lilianne/qucats_predict/data/input_model/STRIPE82-0170_QSOs_VAC_ext.csv", sep=",")

In [35]:
features = create_colors(broad=True, narrow=True, wise=True, galex=True, aper="PStotal")

test.dropna(subset=features)

,ID,RA,DEC,u_PStotal-r_PStotal,g_PStotal-r_PStotal,r_PStotal-i_PStotal,r_PStotal-z_PStotal,J0378_PStotal-r_PStotal,J0395_PStotal-r_PStotal,J0410_PStotal-r_PStotal,...,e_r_PStotal,e_i_PStotal,e_z_PStotal,e_J0378_PStotal,e_J0395_PStotal,e_J0410_PStotal,e_J0430_PStotal,e_J0515_PStotal,e_J0660_PStotal,e_J0861_PStotal
0,iDR4_3_STRIPE82-0170_0000001,358.047743,-0.009816,77.443637,2.317722,0.857072,1.098405,77.443637,0.585045,0.095419,...,0.201302,0.137540,0.207449,99.000000,1.085668,0.643955,0.944720,0.672653,0.327311,0.191052
1,iDR4_3_STRIPE82-0170_0000002,358.127397,-0.009661,1.973071,0.570030,-0.235575,-2.201269,77.171662,1.463084,-0.503849,...,0.257825,0.466810,5.511013,99.000000,3.143161,0.582275,41.537113,99.000000,99.000000,12.149977
2,iDR4_3_STRIPE82-0170_0000003,358.060380,-0.009696,2.318148,1.293744,1.033831,1.531872,1.585646,78.017609,0.568928,...,0.122202,0.071693,0.084828,1.051259,99.000000,0.563642,99.000000,0.719231,0.086034,0.124588
3,iDR4_3_STRIPE82-0170_0000004,358.052612,-0.009731,2.966395,1.772986,0.274541,0.574447,5.902094,78.547750,1.600892,...,0.077325,0.086380,0.122907,34.596550,99.000000,0.855274,0.859392,0.515587,0.082441,0.128695
4,iDR4_3_STRIPE82-0170_0000005,358.165417,-0.009693,2.629145,1.099544,0.485080,0.734758,78.901508,1.125943,0.845308,...,0.058801,0.053760,0.078161,99.000000,0.475480,15.232102,17.248413,12.600427,0.058719,0.097545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48819,iDR4_3_STRIPE82-0170_0048821,357.937728,1.283971,76.383064,0.325888,-0.187680,-0.573120,0.877135,1.263621,76.383064,...,0.475903,0.844066,2.250870,2.142443,4.724686,99.000000,0.903228,99.000000,0.569441,99.000000
48820,iDR4_3_STRIPE82-0170_0048822,359.064244,1.285420,1.231156,1.068781,0.537860,1.379542,77.294628,77.294628,1.581274,...,0.209448,0.185911,0.165338,99.000000,99.000000,2.510211,1.384425,0.530683,0.230285,0.301298
48821,iDR4_3_STRIPE82-0170_0048823,357.973745,1.283776,0.026971,76.378677,-0.221681,1.227469,-0.448966,76.378677,76.378677,...,0.477369,0.876526,0.432897,0.639549,99.000000,99.000000,1.117439,0.376078,0.316642,0.426974
48822,iDR4_3_STRIPE82-0170_0048824,358.546137,1.283260,77.496210,1.671761,0.477172,0.490885,2.411057,77.496210,1.094122,...,0.170902,0.161541,0.305433,3.239678,99.000000,1.320988,6.789623,0.389488,0.206314,0.193671


In [38]:
test["NUVmag-r_PStotal"]

0        77.443637
1        77.171662
2        78.017609
3        78.547750
4        78.901508
           ...    
48819    76.383064
48820    77.294628
48821    76.378677
48822    77.496210
48823    78.720785
Name: NUVmag-r_PStotal, Length: 48824, dtype: float64